In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from pandas import Series, DataFrame
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, ExtraTreesClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
import xam
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import os, random, math, glob
from IPython.display import Image as IM
from IPython.display import clear_output
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]

Using TensorFlow backend.


In [2]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_sub = pd.read_csv("gender_submission.csv")
print(df.shape, df.columns)
df.head(12)

(891, 12) Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [4]:
#df['Cabin'].value_counts()

In [5]:
df.shape

(891, 12)

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [7]:
#y = df['Survived']

In [8]:
# # Plot the distribution
# ax = df['Survived'].plot(kind='hist')
# plt.xlabel('Percentage of households that are w-headed and have income under R19.6k/month')

In [9]:
# # Strong +ve correlations
# df.corr()['Survived'].sort_values(ascending=False).head(4)

In [10]:
# # And negative correlations
# df.corr()['Survived'].sort_values().head(5)           

In [11]:
# # pw_00: running water in the home. Places where almost everyone has piped water tend to be more affluent.
# df.plot(x='PassengerId', y='Survived', kind='scatter', alpha=0.3)

In [12]:
df= df.drop(['PassengerId','Name','Cabin','Ticket'],axis=1)
test = test.drop(['PassengerId','Ticket','Name','Cabin'],axis=1)     

In [16]:
from fancyimpute import KNN #We use the train dataframe from Titanic dataset#fancy impute removes column names.
#train_cols = list(train)# Use 5 nearest rows which have a feature to fill in each row's
# missing featurestrain = pd.DataFrame(KNN(k=5).complete(train))

df = pd.DataFrame(KNN(k=5).complete(df))
#train.columns = train_cols

ModuleNotFoundError: No module named 'fancyimpute'

In [ ]:
uhjjk

In [ ]:
df['Embarked']= df['Embarked'].replace([np.nan],'S')
df['Age']= df['Age'].replace([np.nan],24.00)
#df['Cabin']= df['Cabin'].replace([np.nan],'G6')

test['Embarked']= test['Embarked'].replace([np.nan],'S')
test['Age']= test['Age'].replace([np.nan],24.00)
test['Fare']= test['Fare'].replace([np.nan],7.75)
#test['Cabin']= test['Cabin'].replace([np.nan],'G6')

In [ ]:
df=pd.get_dummies(df, columns=['Embarked'], drop_first=True)
df=pd.get_dummies(df, columns=['Sex'], drop_first=True)

test=pd.get_dummies(test, columns=['Embarked'], drop_first=True)
test=pd.get_dummies(test, columns=['Sex'], drop_first=True)

In [ ]:
y = df['Survived']
train_x = df.drop('Survived', axis=1)

In [ ]:
# sns.countplot(y) #To check if the data is balanced

In [ ]:
# #splitting the training data
# xtrain, xtest, ytrain, ytest = train_test_split(train_x, y, test_size=0.38, random_state=100)

In [ ]:
# sm = SMOTE(random_state=42, ratio = 1.0)
# xresample, yresample = sm.fit_resample(xtrain,ytrain)

In [ ]:
# xtrain = xresample
# ytrain = yresample

In [ ]:
# sns.countplot(ytrain)

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(32,10))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(train_x.corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
jjnkj

In [ ]:
# # Preparing for Mean Encoding
# categ_col = df.select_dtypes(exclude=['int64','float64']).columns
# categ_col = list(categ_col)
# categ_col = test.select_dtypes(exclude=['int64','float64']).columns
# categ_col = list(categ_col)

# numb_col = df.select_dtypes(exclude=['object']).columns
# numb_col = list(numb_col)
# numb_col = test.select_dtypes(exclude=['object']).columns
# numb_col = list(numb_col)


# #Performing Mean Encoding on all the categorical features
# xam_it = xam.feature_extraction.BayesianTargetEncoder(columns=categ_col)

# y = df['Survived']
# train_x = df.drop('Survived', axis=1)
# xam_it.fit(train_x, y)

# train_x = xam_it.transform(train_x)
# test_x = xam_it.transform(test)

# train_x.drop(columns= categ_col, inplace=True)
# test_x.drop(columns= categ_col, inplace=True)

In [ ]:
#splitting the training data
xtrain, xtest, ytrain, ytest = train_test_split(train_x, y, test_size=0.38, random_state=100)

In [ ]:
sfdfg

In [ ]:
# using gridsearch & cross validation, hypertune best parameters 
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
model = RandomForestClassifier(n_estimators=1200,
                               max_depth=15,
                               min_samples_split=5,
                               min_samples_leaf=5,
                               max_features=None,
                               oob_score=True,
                               random_state=42)

grid = {#'n_estimators': [10,20,30,50,80,100],
        #'max_leaf_nodes':[1,0]
        #'min_impurity_split':[1,2,3,4,5,6,7,8,9,10,15,20,30,40,50,60,70,80,90,100],
        #'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,15,20,30,40,50,60,70,80,90,100]
        #'max_features':[10],
        #'min_weight_fraction_leaf':[0,0.5]
        #'random_state':[1,2,3,4,5,6,7,8,9,10,15,20,30,40,50,60,70,80,90,100]
        #'max_depth': [1,2,3,4,5]
         
        #'learning_rate' : [0.5,0.52,0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7,1.5]
}
cv = KFold(n_splits = 10, shuffle=False, random_state = 42)
clf = GridSearchCV(model, grid, n_jobs=8, cv=cv,scoring='neg_mean_squared_error')
clf.fit(xtrain, ytrain)

In [ ]:
clf.best_score_, clf.best_params_

In [ ]:
# Predicting the Test set results
y_pred = clf.predict(xtest)

In [ ]:
classifier_pred = clf.predict(xtest)
print('accuracy_score:' ,accuracy_score(ytest, classifier_pred))

In [ ]:

rf = RandomForestClassifier(n_estimators=100)
rf.fit(train_x,y)
features = train_x
importances = rf.feature_importances_
indices = np.argsort(importances)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
classifier =  RandomForestClassifier(n_estimators=1200,
                               max_depth=15,
                               min_samples_split=5,
                               min_samples_leaf=5,
                               max_features=None,
                               oob_score=True,
                               random_state=42)


classifier.fit(xtrain, ytrain)
classifier_pred = classifier.predict(xtest)
print('accuracy_score:' ,accuracy_score(ytest, classifier_pred))

In [ ]:
classifier = GaussianNB()
classifier.fit(xtrain, ytrain)
classifier_pred = classifier.predict(xtest)
print('accuracy_score:' ,accuracy_score(ytest, classifier_pred))

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear',degree=5)
classifier.fit(xtrain, ytrain)
classifier_pred = classifier.predict(xtest)
print('accuracy_score:' ,accuracy_score(ytest, classifier_pred))

In [ ]:


classifier = GradientBoostingClassifier()
classifier.fit(xtrain, ytrain)
classifier_pred = classifier.predict(xtest)
print('accuracy_score:' ,accuracy_score(ytest, classifier_pred))

In [ ]:
oiyiuhui

In [ ]:

train_x = train_x.as_matrix()
test = test.as_matrix()

In [ ]:
classifier = XGBClassifier()
classifier.fit(xtrain, ytrain)
classifier_pred = classifier.predict(xtest)
print('accuracy_score:' ,accuracy_score(ytest, classifier_pred))


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression

clf3 = GradientBoostingClassifier()
clf4 = LGBMClassifier()
#clf5 = XGBClassifier()
clf6 = GaussianNB()

eclf = VotingClassifier(estimators=[('gbc', clf3),('lgbm',clf4),('cgnb', clf6)], voting='soft')
eclf.fit(xtrain, ytrain)
eclf_pred = eclf.predict(xtest)
print('accuracy_score:' ,accuracy_score(ytest, eclf_pred))

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
clf1 = CatBoostClassifier(depth = 4,random_strength = 2)
#clf2 = AdaBoostClassifier()
clf3 = GradientBoostingClassifier()
clf4 = LGBMClassifier()
#clf5 = XGBClassifier()
clf6 = GaussianNB()
#clf7 = LogisticRegression(solver='lbfgs', multi_class='multinomial',random_state=1)
eclf = VotingClassifier(estimators=[('cbc', clf1),('gbc', clf3),('lgbm',clf4),('gnb',clf6)], voting='soft')
eclf.fit(xtrain, ytrain)
eclf_pred = eclf.predict(xtest)
print('accuracy_score:' ,accuracy_score(ytest, eclf_pred))

In [ ]:

classifier = LGBMClassifier()
classifier.fit(xtrain, ytrain)
classifier_pred = classifier.predict(xtest)
print('accuracy_score:' ,accuracy_score(ytest, classifier_pred))

In [ ]:

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(xtrain, ytrain)
classifier_pred = classifier.predict(xtest)
print('accuracy_score:' ,accuracy_score(ytest, classifier_pred))


In [ ]:
# y = df['Survived']
# train_x = df.drop('Survived', axis=1)

In [ ]:
categorical_features_indices = np.where(train_x.dtypes!=np.float)[0]

In [ ]:
#splitting the training data
xtrain, xtest, ytrain, ytest = train_test_split(train_x, y, test_size=0.33, random_state=100)

In [ ]:
cat_features = categorical_features_indices,

In [ ]:
# # # With XGBoost
xgb = CatBoostClassifier(depth = 4,random_strength = 2)
xgb.fit(xtrain, ytrain,eval_set =(xtest, ytest),plot=True)
xgb_pred = xgb.predict(xtest)
print('accuracy_score:' ,accuracy_score(ytest, xgb_pred))

In [ ]:
# # # # With XGBoost
# xgb = CatBoostClassifier(depth = 5,random_strength = 2)
# xgb.fit(xtrain, ytrain,eval_set =(xtest, ytest),plot=True)
# xgb_pred = xgb.predict(xtest)


In [ ]:
train_x[train_x==np.inf]=np.nan
train_x[train_x==-np.inf]=np.nan
train_x.fillna(train_x.mean(),inplace=True)




test_x[test_x==np.inf]=np.nan
test_x[test_x==-np.inf]=np.nan
test_x.fillna(test_x.mean(),inplace=True)

In [ ]:
# #Predicting the test set
# promopredd= DataFrame(xgb.predict_proba(test)[:,1])
# promopredd=promopredd.rename(columns={0:'Survived'})


In [ ]:
#Predicting the test set
promopredd= DataFrame(clf.predict(test))
promopredd=promopredd.rename(columns={0:'Survived'})


In [ ]:
aasa =  pd.read_csv('test.csv')
vom = aasa[['PassengerId']]

In [ ]:
sample_suball=pd.concat([vom, promopredd], axis=1)
prediction=sample_suball.to_csv('kkddbjii.csv' , index=False, index_label=None)